Pulp

In [ ]:
!pip install pulp

In [ ]:
import pulp
import pandas as pd

## Maximization of Reward 


In [ ]:
df = pd.DataFrame({"Train": ["x91"],
                   "SplitCode": [21],
                   "2021-09-30": [100],
                   "2021-10-01": [50],
                   "2021-10-02": [1000]})
df

,Train,SplitCode,2021-09-30,2021-10-01,2021-10-02
0,x91,21,100,50,1000


In [ ]:
choose_or_not_date = ["2021-09-30", "2021-10-01", "2021-10-02"]
choose_or_not_date

['2021-09-30', '2021-10-01', '2021-10-02']

In [ ]:
prob = pulp.LpProblem("Reward_Maintenance", pulp.LpMaximize)
prob

Reward_Maintenance:
MAXIMIZE
None
VARIABLES

In [ ]:
x = pulp.LpVariable.dicts("x", choose_or_not_date, 0, 1, pulp.LpInteger)

In [ ]:
x

{'2021-09-30': x_2021_09_30,
 '2021-10-01': x_2021_10_01,
 '2021-10-02': x_2021_10_02}

In [ ]:
reward = [100, 50, 1000]

In [ ]:
prob += pulp.lpSum(
    [x[date] * reward_date for date in choose_or_not_date for reward_date in reward])

In [ ]:
prob

Reward_Maintenance:
MAXIMIZE
1150*x_2021_09_30 + 1150*x_2021_10_01 + 1150*x_2021_10_02 + 0
VARIABLES
0 <= x_2021_09_30 <= 1 Integer
0 <= x_2021_10_01 <= 1 Integer
0 <= x_2021_10_02 <= 1 Integer

In [ ]:
# constraints
prob += pulp.lpSum(
    [x[date] for date in choose_or_not_date]) <= 1
prob

Reward_Maintenance:
MAXIMIZE
1150*x_2021_09_30 + 1150*x_2021_10_01 + 1150*x_2021_10_02 + 0
SUBJECT TO
_C1: x_2021_09_30 + x_2021_10_01 + x_2021_10_02 <= 1

VARIABLES
0 <= x_2021_09_30 <= 1 Integer
0 <= x_2021_10_01 <= 1 Integer
0 <= x_2021_10_02 <= 1 Integer

In [ ]:
%time prob.solve()

CPU times: user 1.48 ms, sys: 4.11 ms, total: 5.59 ms
Wall time: 21 ms


1

In [ ]:
print(pulp.LpStatus[prob.status])

Optimal


In [ ]:
for v in prob.variables():
    print(v, v.varValue)

x_2021_09_30 0.0
x_2021_10_01 0.0
x_2021_10_02 1.0


## Wedding Tables Assignment

Maximise the total happiness of all of the tables.

In [ ]:
#create list of all possible tables
max_tables = 5
max_table_size = 4
guests = 'A B C D E F G I J K L M N O P Q R'.split()

possible_tables = [tuple(c) for c in pulp.allcombinations(guests, 
                                        max_table_size)]
#possible_tables

In [ ]:
#create a binary variable to state that a table setting is used
x = pulp.LpVariable.dicts('table', possible_tables, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

In [ ]:
# x

In [ ]:
# objective function
seating_model = pulp.LpProblem("Wedding Seating Model", pulp.LpMinimize)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
seating_model

Wedding_Seating_Model:
MINIMIZE
None
VARIABLES

In [ ]:
def happiness(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(ord(table[0]) - ord(table[-1]))

In [ ]:
seating_model += pulp.lpSum([happiness(table) * x[table] for table in possible_tables])

In [ ]:
# seating_model

In [ ]:
#specify the maximum number of tables
seating_model += pulp.lpSum([x[table] for table in possible_tables]) <= max_tables, \
                            "Maximum_number_of_tables"

In [ ]:
#A guest must seated at one and only one table
for guest in guests:
    seating_model += pulp.lpSum([x[table] for table in possible_tables
                                if guest in table]) == 1, "Must_seat_%s"%guest

In [ ]:
seating_model.solve()

1

In [ ]:
print("The choosen tables are out of a total of %s:" %len(possible_tables))
for table in possible_tables:
    if x[table].value() == 1.0:
        print(table)

The choosen tables are out of a total of 3213:
('M', 'N')
('E', 'F', 'G')
('A', 'B', 'C', 'D')
('I', 'J', 'K', 'L')
('O', 'P', 'Q', 'R')


http://www.optimization-online.org/DB_FILE/2011/09/3178.pdf

In [ ]:
require = {1:7, 2:5, 3:3, 4:2, 5:2}
products = [1, 2, 3, 4, 5]
locations = [1, 2, 3, 4, 5]
capacity = 8

In [ ]:
prob = pulp.LpProblem("FacilityLocation", pulp.LpMinimize)

In [ ]:
use_vars = pulp.LpVariable.dicts("UseLocation", locations, 0, 1, pulp.LpBinary)
use_vars

In [ ]:
waste_vars = pulp.LpVariable.dicts("Waste", locations, 0, capacity)
waste_vars

In [ ]:
assign_vars = pulp.LpVariable.dicts("AtLocation", 
                                    [(i, j) for i in locations
                                            for j in products],
                                    0, 1, pulp.LpBinary)
assign_vars

In [ ]:
# objective function
prob += pulp.lpSum(waste_vars[i] for i in locations)
print(prob)

In [ ]:
for j in products:
  prob += pulp.lpSum(assign_vars[(i, j)] for i in locations) == 1
print(prob)

It’s worth mentioning that almost all widely used linear programming and mixed-integer linear programming libraries are native to and written in Fortran or C or C++. This is because linear programming requires computationally intensive work with (often large) matrices. Such libraries are called solvers. The Python tools are just wrappers around the solvers.

Several free Python libraries are specialized to interact with linear or mixed-integer linear programming solvers:

- SciPy Optimization and Root Finding  
- PuLP  
- Pyomo  
- CVXOPT

## Small Linear Programming Problem

MAX z = x + 2y  
subject to  
$2x + y \leq 20$  
$-4x + 5y \leq 10$  
$-x +2y \geq -2$  
$x \geq 0$  
$y \geq 0$  

x, y - decision variables  
the function of decision variables to be maximized/minimized, z - the objective funciton /the cost function  
constraints - inequality  
--> feasible region - feasible solutions  

You want to maximize z. The feasible solution that corresponds to maximal z is the *optimal solution*. 


A linear programming problem is infeasible if it does not have a solution. This usually happens when no solution can satisfy all constraints at once.

A linear programming problem is unbounded if its feasible region isn't bounded and the solution is not finite.

Dropping constraints out of a problem --> relaxing the problem.

## Resource Allocation Problem

Say that a factory produces four different products, and that the daily produced amount of the first product is x₁, the amount produced of the second product is x₂, and so on. The goal is to determine the profit-maximizing daily production amount for each product, bearing in mind the following conditions:

1. The profit per unit of product is $20, $12, $40, and $25 for the first, second, third, and fourth product, respectively.

2. Due to manpower constraints, the total number of units produced per day can’t exceed fifty.

3. For each unit of the first product, three units of the raw material A are consumed. Each unit of the second product requires two units of the raw material A and one unit of the raw material B. Each unit of the third product needs one unit of A and two units of B. Finally, each unit of the fourth product requires three units of B.

4. Due to the transportation and storage constraints, the factory can consume up to one hundred units of the raw material A and ninety units of B per day.

### Using Scipy

for small problems

In [ ]:
from scipy.optimize import linprog

*Example*

MIN -z = -x -2y  
subject to  
$2x + y \leq 20$  
$-4x + 5y \leq 10$  
$x -2y \leq 2$  
$-x + 5y = 15$  
$x \geq 0$  
$y \geq 0$ 

In [ ]:
obj = (-1, -2)  # coefficients for x and y
lhs_ineq = [[2, 1],
            [-4, 5],
            [1, -2]]
lhs_eq = [[-1, 5]]
rhs_ineq = [20,
            10,
            2]
rhs_eq = [15]

In [ ]:
bnd = [(0, float("inf")),    # bounds of x, y
       (0, float("inf"))]

The statement above is redundant because linprog() takes these bounds (zero to positive infinity) by default.

In [ ]:
opt = linprog(c = obj, A_ub = lhs_ineq, b_ub = rhs_ineq, A_eq = lhs_eq, b_eq = rhs_eq, bounds = bnd, method = "revised simplex")
opt

c : the coefficients from the objective function  
A_ub, b_ub : the coefficients from the left and right sides of the  inequality constraints.  

method = "interior-point : selects the interior-point method. Default.

method = "revised simplex" : the revised two-phase simplex method.

method = "simplex" : two-phase simplex method.


In [ ]:
print(opt.fun)
print(opt.x)

*Example*  
MAX $20x_1 + 12x_2 + 40x_3 + 25x_4$ (profit)  
s.t.  
$x_1 + x_2 + x_3 + x_4 \leq 50$  (manpower)  
$3x_1 + 2x_2 + x_3 \leq 100$  (material A)  
$x_2 + 2x_3 + 3x_4 \leq 90$  (material B)  
$x_1, x_2, x_3, x_4 \geq 0$



In [ ]:
obj = [-20, -12, -40, -25]
lhs_ineq = [[1, 1, 1, 1],
            [3, 2, 1, 0],
            [0, 1, 2, 3]]
rhs_ineq = [50, 100, 90]
opt = linprog(c = obj, A_ub = lhs_ineq, b_ub = rhs_ineq, method = "revised simplex")
opt

The first slack = 0: the values of the left and right sides of the manpower constraint are the same. The factory produces 50 units per day and this is its full capacity.  
The second slack = 40: the factory consumes 60 unites of raw material A out of a potential 100 units.  
The third slack = 0: the factory consumes all 90 units of the raw material B. This is why the factory can not produce the second or fourth product at all, can't produce more than 45 unites of the third product. It lacks the raw material B.

In [ ]:
### Using PulP

In [ ]:
pip install Pulp

## Example - Resourcing Problem

We are consulting for a boutique car manufacturer, producing luxury cars.  
They run on one month (30 days) cycles, we have one cycle to show we can provide values.  
There is one robot, two engineers, one detailer in the factory. The detailer has some holiday off, so only has 21 days available.   
The 2 cars need different time with each resource:  
Robot time: car A = 3 days, car B = 4 days  
Engineer time : car A = 5 days, car B = 6 days  
Detailer time : car A = 1.5 days, car B = 3 days  
Car A provides $30,000 profit, Car B $45, 000.  
At the moment, they produce 4 of each car per month, for 300,000 dollar profit. Not bad at all, but we think we can do better for them.

a = number of car A  
b = number of car B  

Model:  
MAX profit = a 30000 + b 45000  
subject to  
$a \leq 0$  
$b \leq 0$  
$3 a + 4 b \leq 30$  
$5 a + 6 b \leq 60$  # 2 engineers  
$1.5 a + 3 b \leq 21$




In [ ]:
import pulp

In [ ]:
model = pulp.LpProblem("Profit maximising problem", pulp.LpMaximize)

The decision variables are integer because we cannot sell half a car.

In [ ]:
a = pulp.LpVariable('a', lowBound=0, cat='Integer')
b = pulp.LpVariable('b', lowBound=0, cat='Integer')

In [ ]:
# objective function
model += 30000*a + 45000*b, "Profit"

# constraints
model += 3*a + 4*b <= 30
model += 5*a + 6*b <= 60
model += 1.5*a + 3*b <= 21

In [ ]:
# solve our problem
model.solve()
pulp.LpStatus[model.status]

In [ ]:
# print our decision variable values
print("Production of Car A = {}".format(a.varValue))
print("Production of Car B = {}".format(b.varValue))

In [ ]:
# print our objective function value
pulp.value(model.objective)

## Example - Making Sausages



In [ ]:
import pandas as pd


In [ ]:
data = {'Ingredient': ['Pork', 'Wheat', 'Starch'], 
        'Cost (euro/kg)': [4.32, 2.46, 1.86],
        'Availability (kg)': [30, 20, 17]}
pd.DataFrame(data)

We will make 2 types of sausages:  
- Economy (>40% pork)  
- Premium (>60% pork)

One sausage is 50g (0.05kg)

According to government regulations, the most starch we can use in our sausage is 25%. We have a contract with a butcher, and have already purchased 23 kg pork, that will go bad if it's not used.  

We have a demand for 350 economy sausages, 500 premium sausages.

We need to figure out how to most cost effectively blend our sausages.

P_e = pork in the economy sausages (kg)  
W_e = wheat in the economy sausages (kg)
S_e = starch in the economy sausages (kg)
P_p = pork in the premium sausages (kg)  
W_p = wheat in the premium sausages (kg)
S_p = starch in the premium sausages (kg)

In [ ]:
# import pulp
model = pulp.LpProblem("Cost minimising blending problem", pulp.LpMinimize)

6 decision variables.  
We could name them individually but this would not scale up if we have hundreds/thousands of variables.  
We will create a couple of lists from which we can create tuple indices.

In [ ]:
sausage_types = ['economy', 'premium']
ingredients = ['pork', 'wheat', 'starch']

Each of these decision variables have similar characteristics: lower bound of 0, continuous variables). Therefore, we can use PuLP's LpVariable object's dict functionality, we can provide our tuple indices.

In [ ]:
ing_weight = pulp.LpVariable.dicts('weight kg',
                                   ((i, j) for i in sausage_types for j in ingredients),
                                   lowBound = 0,
                                   cat = 'Countinuous')
ing_weight

PuLP provides an lpSum vector calculation for the sum of a list of linear expressions.  
Whilst we only have 6 decision variables, I will demonstrate how the problem would be constructed in a way that could be scaled up to many variables using list comprehensions.

In [ ]:
model += (pulp.lpSum([
                      4.32*ing_weight[(i, 'pork')] + 2.46*ing_weight[(i, 'wheat')] + 1.86*ing_weight[(i, 'starch')]
                      for i in sausage_types])
)

In [ ]:
# Constraints
# 350 economy and 500 premium sausages at 0.05kg
model += pulp.lpSum([ing_weight['economy', j] for j in ingredients]) == 350*0.05
model += pulp.lpSum([ing_weight['premium', j] for j in ingredients]) == 500*0.05

# economy has >= 40% pork, premium >= 60% pork
model += ing_weight['economy', 'pork'] >= (0.4 * pulp.lpSum([ing_weight['economy', j] for j in ingredients]))
model += ing_weight['premium', 'pork'] >= (0.6 * pulp.lpSum([ing_weight['premium', j] for j in ingredients]))

# sausage must be <= 25% starch
model += ing_weight['economy', 'starch'] <= 0.25 * pulp.lpSum([ing_weight['economy', j] for j in ingredients])
model += ing_weight['premium', 'starch'] <= 0.25 * pulp.lpSum([ing_weight['premium', j] for j in ingredients])

# We have at most 30 kg of pork, 20 kg of wheat and 17 kg of starch available
model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) <= 30
model += pulp.lpSum([ing_weight[i, 'wheat'] for i in sausage_types]) <= 20
model += pulp.lpSum([ing_weight[i, 'starch'] for i in sausage_types]) <= 17

# We have at least 23 kg of pork to use up
model += pulp.lpSum([ing_weight[i, 'pork'] for i in sausage_types]) >= 23

In [ ]:
# solve our problem
model.solve()
pulp.LpStatus[model.status]

In [ ]:
for var in ing_weight:
    var_value = ing_weight[var].varValue
    print("The weight of {0} in {1} sausages is {2} kg".format(var[1], var[0], var_value))

In [ ]:
total_cost = pulp.value(model.objective)

print("The total cost is €{} for 350 economy sausages and 500 premium sausages".format(round(total_cost, 2)))

## Balanced Diet Problem Simple